In [1]:
import heapq
import random
import time
from collections import deque

OBJETIVO = (1, 2, 3, 4, 5, 6, 7, 8, 0)
MOVS = {'arriba': -3, 'abajo': 3, 'izq': -1, 'der': 1}
COORD = {i: (i // 3, i % 3) for i in range(9)}

def es_resoluble(estado):
    inversions = 0
    lista = [n for n in estado if n != 0]
    for i in range(len(lista)):
        for j in range(i + 1, len(lista)):
            if lista[i] > lista[j]:
                inversions += 1
    return inversions % 2 == 0

def generar_estado_aleatorio():
    while True:
        estado = tuple(random.sample(range(9), 9))
        if es_resoluble(estado):
            return estado

def funcion_sucesor(estado):
    sucesores = []
    i = estado.index(0)
    for mov, delta in MOVS.items():
        j = i + delta
        if mov == 'arriba' and i >= 3:
            pass
        elif mov == 'abajo' and i < 6:
            pass
        elif mov == 'izq' and i % 3 != 0:
            pass
        elif mov == 'der' and i % 3 != 2:
            pass
        else:
            continue
        nuevo = list(estado)
        nuevo[i], nuevo[j] = nuevo[j], nuevo[i]
        sucesores.append(tuple(nuevo))
    return sucesores

def reconstruir_camino(parent):
    camino = []
    cur = OBJETIVO
    while cur is not None:
        camino.append(cur)
        cur = parent[cur]
    return list(reversed(camino))

def bfs_puzzle(inicial):
    inicio_t = time.time()
    nodos = 0
    cola = deque([inicial])
    visitados = set([inicial])
    parent = {inicial: None}
    while cola:
        estado = cola.popleft()
        nodos += 1
        if estado == OBJETIVO:
            return reconstruir_camino(parent), nodos, time.time() - inicio_t
        for sucesor in funcion_sucesor(estado):
            if sucesor not in visitados:
                visitados.add(sucesor)
                parent[sucesor] = estado
                cola.append(sucesor)
    return None, nodos, time.time() - inicio_t

def dfs_puzzle(inicial, limite=40):
    inicio_t = time.time()
    nodos = 0
    stack = [(inicial, 0)]
    parent = {inicial: None}
    visitados = set([inicial])
    while stack:
        estado, prof = stack.pop()
        nodos += 1
        if estado == OBJETIVO:
            return reconstruir_camino(parent), nodos, time.time() - inicio_t
        if prof < limite:
            for sucesor in funcion_sucesor(estado):
                if sucesor not in visitados:
                    visitados.add(sucesor)
                    parent[sucesor] = estado
                    stack.append((sucesor, prof + 1))
    return None, nodos, time.time() - inicio_t

def ucs_puzzle(inicial):
    inicio_t = time.time()
    nodos = 0
    heap = [(0, inicial)]
    parent = {inicial: None}
    dist = {inicial: 0}
    visitados = set()
    while heap:
        costo, estado = heapq.heappop(heap)
        nodos += 1
        if estado in visitados:
            continue
        visitados.add(estado)
        if estado == OBJETIVO:
            return reconstruir_camino(parent), nodos, time.time() - inicio_t
        for sucesor in funcion_sucesor(estado):
            nuevo_costo = costo + 1
            if sucesor not in dist or nuevo_costo < dist[sucesor]:
                dist[sucesor] = nuevo_costo
                parent[sucesor] = estado
                heapq.heappush(heap, (nuevo_costo, sucesor))
    return None, nodos, time.time() - inicio_t

def heuristica_propia(estado):
    h = 0
    for idx, valor in enumerate(estado):
        if valor == 0:
            continue
        x1, y1 = COORD[idx]
        x2, y2 = COORD[valor]
        h += abs(x1 - x2) + abs(y1 - y2)
        if x1 == x2:
            h += 2
        if y1 == y2:
            h += 2
    return h

def a_estrella_puzzle(inicial):
    inicio_t = time.time()
    nodos = 0
    heap = [(heuristica_propia(inicial), 0, inicial)]
    parent = {inicial: None}
    g_cost = {inicial: 0}
    visitados = set()
    while heap:
        f, g, estado = heapq.heappop(heap)
        nodos += 1
        if estado in visitados:
            continue
        visitados.add(estado)
        if estado == OBJETIVO:
            return reconstruir_camino(parent), nodos, time.time() - inicio_t
        for sucesor in funcion_sucesor(estado):
            nuevo_g = g + 1
            if sucesor not in g_cost or nuevo_g < g_cost[sucesor]:
                g_cost[sucesor] = nuevo_g
                parent[sucesor] = estado
                f_nuevo = nuevo_g + heuristica_propia(sucesor)
                heapq.heappush(heap, (f_nuevo, nuevo_g, sucesor))
    return None, nodos, time.time() - inicio_t

estado_inicial = generar_estado_aleatorio()
print("Estado inicial:")
for i in range(0, 9, 3):
    print(estado_inicial[i:i+3])

print("\nEjecutando BFS...")
bfs_res = bfs_puzzle(estado_inicial)

print("Ejecutando DFS...")
dfs_res = dfs_puzzle(estado_inicial)

print("Ejecutando UCS...")
ucs_res = ucs_puzzle(estado_inicial)

print("Ejecutando A*...")
a_res = a_estrella_puzzle(estado_inicial)

print("\nResumen de resultados:")
print("BFS:", bfs_res[1], "nodos,", bfs_res[2], "segundos")
print("DFS:", dfs_res[1], "nodos,", dfs_res[2], "segundos")
print("UCS:", ucs_res[1], "nodos,", ucs_res[2], "segundos")
print("A*:", a_res[1], "nodos,", a_res[2], "segundos")

Estado inicial:
(1, 4, 6)
(0, 5, 3)
(8, 2, 7)

Ejecutando BFS...
Ejecutando DFS...
Ejecutando UCS...
Ejecutando A*...

Resumen de resultados:
BFS: 121335 nodos, 0.2640092372894287 segundos
DFS: 101138 nodos, 0.15714144706726074 segundos
UCS: 103929 nodos, 0.4019014835357666 segundos
A*: 99926 nodos, 0.6086232662200928 segundos
